<a href="https://colab.research.google.com/github/FlavioSarto/FlavioSarto/blob/main/Trilha_4_AP_flavios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Tarefa de Aprofundamento 3** 
## CASO: Controle de Qualidade de Peças de uma Indústria
---
<br>
<br>





A partir deste notebook você vai criar e avaliar diferentes modelos de classificação para prever a qualidade de peças de uma indústria. 

Em seguida, com base nos seus resultados, responda o **questionário do Moodle**. (**dica**: o questionário é de alternativas e as respontas irão ajudá-lo a verificar os seus resultados ao longo do programa). 

# Basic imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Exploração dos Dados



`pieces` é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

**Objetivo:** Seu objetivo é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. 



In [2]:
df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces1.csv')
df.head()

,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ


## Inspecione os dados

In [3]:
# seu código
display(df)
df.describe(include='all')
df.isnull().sum()/len(df)

,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ
...,...,...,...,...,...,...,...
495,155,4.6,3.6,1.0,0.2,Reject,SP
496,120,6.9,3.2,5.7,2.3,Refurbish,RJ
497,321,6.7,3.1,5.6,2.4,Refurbish,RJ
498,448,5.6,3.0,4.1,1.3,Accept,SP


id         0.000
A          0.000
B          0.018
C          0.012
D          0.008
Quality    0.000
Unit       0.000
dtype: float64

## Preparação dos dados

Aqui você vai preparar o dado para o uso nos modelos. Você fará a seleção de atributos, tratamento de valores nulos, o encode dos dados, normalização e a separação dos dados de treinamento e teste.

### Seleção de Atributos

Elimine atributos que não fazem parte do treinamento.

### Tratamento de Dados Ausentes

Verifique as presença de valores ausentes e, se houverem, subtitua pelo valor médio dos atributos.

In [4]:
# Testa se há dados ausentes
display(df.isnull().sum())
display(df.isnull().sum()/len(df))

id         0
A          0
B          9
C          6
D          4
Quality    0
Unit       0
dtype: int64

id         0.000
A          0.000
B          0.018
C          0.012
D          0.008
Quality    0.000
Unit       0.000
dtype: float64

In [5]:
# Dados ausentes para as colunas B, C e D preenchidos com o valor médio
df['B'] = df[['B']].fillna(df['B'].mean())
df['C'] = df[['C']].fillna(df['C'].mean())
df['D'] = df[['D']].fillna(df['D'].mean())

In [6]:
# Testa novamente se há dados ausentes
display(df.isnull().sum())
display(df.isnull().sum()/len(df))

id         0
A          0
B          0
C          0
D          0
Quality    0
Unit       0
dtype: int64

id         0.0
A          0.0
B          0.0
C          0.0
D          0.0
Quality    0.0
Unit       0.0
dtype: float64

### Hot encode

Faça os *Hot Encodes* **necessários** dos dados. Lembre-se de verificar se de fato todos os dados categóricos precisam dessa transformação.

In [7]:
# seu código
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore')
hot_encode = hot_encode.fit(df[['Unit']])

display(hot_encode.categories_)

df = pd.concat([df, pd.DataFrame(hot_encode.transform(df[['Unit']]).toarray(),columns=hot_encode.categories_)],axis=1)

df = df.drop(columns=['id','Unit'])
df.columns=['A','B','C','D','Quality','BH','RJ','SP']

#df = df.drop(columns=['Unit'])
#df.columns=['id','A','B','C','D','Quality','BH','RJ','SP']
df.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,Quality,BH,RJ,SP
0,4.9,3.1,1.5,0.1,Reject,0.0,0.0,1.0
1,4.8,3.4,1.6,0.2,Reject,0.0,0.0,1.0
2,6.7,2.5,5.8,1.8,Refurbish,0.0,1.0,0.0
3,7.6,3.0,6.6,2.1,Refurbish,0.0,1.0,0.0
4,4.9,3.1,1.5,0.1,Reject,0.0,1.0,0.0


In [8]:
display(df.columns)
df
df.describe(include='all')

Index(['A', 'B', 'C', 'D', 'Quality', 'BH', 'RJ', 'SP'], dtype='object')

,A,B,C,D,Quality,BH,RJ,SP
count,500.000000,500.000000,500.000000,500.000000,500,500.000000,500.000000,500.000000
unique,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,Refurbish,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,182,NaN,NaN,NaN
mean,5.829600,3.076782,3.787449,1.235282,NaN,0.302000,0.250000,0.448000
std,0.817227,0.430583,1.739441,0.769447,NaN,0.459585,0.433446,0.497787
min,4.300000,2.000000,1.000000,0.100000,NaN,0.000000,0.000000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,NaN,0.000000,0.000000,0.000000
50%,5.850000,3.000000,4.400000,1.300000,NaN,0.000000,0.000000,0.000000
75%,6.400000,3.400000,5.100000,1.800000,NaN,1.000000,0.250000,1.000000


### Normalize os dados

Normalize os dados com o `StandardScaler` (melhor) ou a função `scale`.

$$ z_i = \frac{x_i - \bar{x}}{\sigma(x)}$$

E não esqueça de excluir dados não úteis ao treinamento. Depois de normalizar os dados os valores devem apresentar média próxima de 0 e desvio padrão próximo de 1.

In [9]:
# seu código
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df.drop(columns=['Quality'])) # drop somente para o fit, não retirou do data frame
X = scaler.transform(df.drop(columns=['Quality']))

X[0:5]
#X[0][2]

array([[-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
        -0.57735027,  1.1100193 ],
       [-1.26113242,  0.7514042 , -1.25881814, -1.34683562, -0.65777257,
        -0.57735027,  1.1100193 ],
       [ 1.06613215, -1.34088009,  1.1581686 ,  0.73466146, -0.65777257,
         1.73205081, -0.90088523],
       [ 2.16852062, -0.17849993,  1.61854703,  1.12494216, -0.65777257,
         1.73205081, -0.90088523],
       [-1.13864481,  0.05397611, -1.31636544, -1.47692919, -0.65777257,
         1.73205081, -0.90088523]])

(Moodle) **Q1**. Após todas as transformações a soma dos valores absolutos ( `np.abs()` ) dos atributos referentes ao hot encode dos *BH*, *RJ* e *SP* são respectivamente?

*Empregue esta pergunta como ponto de checagem das suas transformações. Se não encontrar os seus valores dentre as alternativas há provavelmente um erro e você deve revisar as transformações.*

In [10]:
# seu código
display(round(np.abs(X[:,4]).sum(),2))
display(round(np.abs(X[:,5]).sum(),2))
display(round(np.abs(X[:,6]).sum(),2))

459.13

433.01

497.29

In [11]:
#.
#59.1, 33.0, 97.2

#b.
#9.1, 3.0, 7.2

#c.
#49.1, 43.0, 47.2

#d.
#149.1, 143.0, 147.2

#e.
#459.1, 433.0, 497.2   ### <===== Esta aqui.

### Conjuntos de Treinamento e Teste

Separe os dados de treinamento e teste (30%, estratificados pela variável objetivo e com random_state = 1984). Não empregue outros parâmetros não solicitados.


In [12]:
# seu código

from sklearn.model_selection import train_test_split

X = X
y = df.Quality

#X_train, X_test, y_train, y_test = ... random_state=1984)
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=1984)

print( X_train.shape, X_test.shape, y_train.shape, y_test.shape )

(350, 7) (150, 7) (350,) (150,)


# K-Vizinhos mais Próximos

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para encontrar o melhor modelo entre os valores de $k$ de $4$ a $11$ e empregando as métricas euclidiana e manhattan. Verifique o modelo obtidos e o `classification_report` desse modelo.

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [13]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [14]:
# seu código

from sklearn import neighbors
from sklearn.neighbors import DistanceMetric
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

base_estimator = neighbors.KNeighborsClassifier()
#param_grid = ...
param_grid = {'n_neighbors':[4,5,6,7,8,9,10,11], 'metric':['euclidean','manhattan']}

#clf = GridSearchCV( ... )
clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
#clf.fit(...)
clf.fit(X_train,y_train)

#print(clf.best_estimator_)
print("Best estimator: ",clf.best_estimator_)

print()
print("Detailed classification report:")
print()
#y_pred = clf.predict( ... )
y_pred=clf.predict(X_test)
#print(classification_report( ... ))
print(classification_report(y_test, y_pred))
print()

Best estimator:  KNeighborsClassifier(metric='manhattan', n_neighbors=7)

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.88      0.98      0.93        47
   Refurbish       0.98      0.89      0.93        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.95       150
   macro avg       0.95      0.96      0.95       150
weighted avg       0.96      0.95      0.95       150




In [15]:
# seu código

from sklearn import neighbors
from sklearn.neighbors import DistanceMetric
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

base_estimator = neighbors.KNeighborsClassifier()
param_grid = {'n_neighbors':[4,5,6,7,8,9,10,11], 'metric':['euclidean','manhattan']}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train,y_train)

print("Best estimator: ",clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

Best estimator:  KNeighborsClassifier(metric='manhattan', n_neighbors=7)

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.88      0.98      0.93        47
   Refurbish       0.98      0.89      0.93        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.95       150
   macro avg       0.95      0.96      0.95       150
weighted avg       0.96      0.95      0.95       150




(Moodle) **Q2** Qual o melhor modelo de Knn obtido e qual a sua acuracidade? 

(Moodle) **Q3** Qual classe teve os elementos mais erroneamente classificados? 

# Regressão Logística

Empregue o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar o modelo de regressão logística. Empregue somente os parâmetros padrão do `sciki-learn` para regressão logística, para isso basta empregar `param_grid = {}`. Verifique os resultados obtidos desse modelo com o `classification_report`. 

Você pode querer empregar o modelo de código no final desse notebook que faz a mesma construção para Árvores de Decisão.

In [16]:
from sklearn.linear_model import LogisticRegression
LogisticRegression().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [17]:
# seu código

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(
    X,y,stratify=y,test_size=0.3,random_state=123)

base_estimator = neighbors.KNeighborsClassifier()
param_grid = {}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train,y_train)

print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

KNeighborsClassifier()

Detailed classification report:

              precision    recall  f1-score   support

      Accept       0.96      0.96      0.96        47
   Refurbish       0.96      0.96      0.96        55
      Reject       1.00      1.00      1.00        48

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150




(Moodle) **Q4** Qual a acuracidade do modelo de regressão logística e qual classe teve mais falsos positivos? 

# Aplicando o melhor Modelo

Empregue o melhor modelo obtido acima para estimar a qualidade das peças das 10 peças do conjunto abaixo.

In [18]:
df_new = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces_new.csv')
display(df_new)

,id,A,B,C,D,Unit
0,182,5.5,2.6,4.4,1.2,SP
1,345,5.8,2.7,4.1,1.0,SP
2,42,5.1,3.5,1.4,0.2,SP
3,37,6.3,2.5,4.9,1.5,BH
4,61,6.0,3.0,4.8,1.8,RJ


## Inspecione os dados

In [19]:
# seu código
display(df_new)

,id,A,B,C,D,Unit
0,182,5.5,2.6,4.4,1.2,SP
1,345,5.8,2.7,4.1,1.0,SP
2,42,5.1,3.5,1.4,0.2,SP
3,37,6.3,2.5,4.9,1.5,BH
4,61,6.0,3.0,4.8,1.8,RJ


In [20]:
df_new.isnull().sum()/len(df_new)

id      0.0
A       0.0
B       0.0
C       0.0
D       0.0
Unit    0.0
dtype: float64

In [21]:
df_new.describe(include='all')

,id,A,B,C,D,Unit
count,5.000000,5.000000,5.000000,5.000000,5.00000,5
unique,NaN,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,NaN,SP
freq,NaN,NaN,NaN,NaN,NaN,3
mean,133.400000,5.740000,2.860000,3.920000,1.14000,NaN
std,132.311375,0.461519,0.403733,1.444645,0.60663,NaN
min,37.000000,5.100000,2.500000,1.400000,0.20000,NaN
25%,42.000000,5.500000,2.600000,4.100000,1.00000,NaN
50%,61.000000,5.800000,2.700000,4.400000,1.20000,NaN
75%,182.000000,6.000000,3.000000,4.800000,1.50000,NaN


## Preparação dos Dados

Lembre-se, as mesmas operações feitas com os dados de treinamento precisarão ser feitas aqui também. 

### Seleção de Atributos 

In [22]:
# seu código

#df_new = df_new.drop(columns= ... )

######## IMPORTANTE ###########
# DROP DA COLUNA id e Unit efetuados no trecho de código a seguir, no hot encode

### Hot encode  

In [23]:
# seu código   (ORIGINAL, SUGERIDO PELO PROFESSOR)

#hot_encode = hot_encode.fit( ... )

#display(hot_encode.categories_)

#df_new = pd.concat( [ df_new.drop(columns='Unit'), pd.DataFrame(hot_encode.transform(df_new[['Unit']]).toarray(), columns = list( hot_encode.categories_[0] )) ], axis=1 ) 
#df_new.head()



# seu código (REALIZADO CONFORME O PRIMEIRO DATASET)
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore')
hot_encode = hot_encode.fit(df_new[['Unit']])

display(hot_encode.categories_)

df_new = pd.concat([df_new, pd.DataFrame(hot_encode.transform(df_new[['Unit']]).toarray(),columns=hot_encode.categories_)],axis=1)

df_new = df_new.drop(columns=['id','Unit'])
df_new.columns=['A','B','C','D','BH','RJ','SP']

#df = df.drop(columns=['Unit'])
#df.columns=['id','A','B','C','D','Quality','BH','RJ','SP']
df_new.head()

[array(['BH', 'RJ', 'SP'], dtype=object)]

,A,B,C,D,BH,RJ,SP
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0


### Normalize os dados

In [24]:
# seu código
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#scaler.fit(df.drop(columns=['Quality'])) # drop somente para o fit, não retirou do data frame
scaler.fit(df_new)
X_new = scaler.transform(df_new)

X_new
#X[0][2]

array([[-0.5814019 , -0.72000341,  0.37147964,  0.11058147, -0.5       ,
        -0.5       ,  0.81649658],
       [ 0.14535047, -0.44307902,  0.13930487, -0.25802342, -0.5       ,
        -0.5       ,  0.81649658],
       [-1.55040507,  1.77231608, -1.95026813, -1.73244298, -0.5       ,
        -0.5       ,  0.81649658],
       [ 1.35660443, -0.9969278 ,  0.7584376 ,  0.6634888 ,  2.        ,
        -0.5       , -1.22474487],
       [ 0.62985206,  0.38769414,  0.68104601,  1.21639614, -0.5       ,
         2.        , -1.22474487]])

## Predição

In [25]:
# seu código

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

base_estimator = neighbors.KNeighborsClassifier()
param_grid = {'n_neighbors':[7], 'metric':['manhattan']}

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=1984)

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

df_new['Prediction'] = clf.predict(X_new)

df_new

,A,B,C,D,BH,RJ,SP,Prediction
0,5.5,2.6,4.4,1.2,0.0,0.0,1.0,Accept
1,5.8,2.7,4.1,1.0,0.0,0.0,1.0,Accept
2,5.1,3.5,1.4,0.2,0.0,0.0,1.0,Reject
3,6.3,2.5,4.9,1.5,1.0,0.0,0.0,Accept
4,6.0,3.0,4.8,1.8,0.0,1.0,0.0,Refurbish


(Moodle) **Q5** Quais as classes obtidas com o melhor modelo para os cinco novos casos? 

# Apêndice: Árvore de Decisão (RESOLVIDO)

Este é apenas um exemplo empregando o `GridSearchCV` com 5 partições e `scoring='accuracy'` para avaliar um modelo de Árvore de Decisão com diferentes parâmetros. Você irá estudar mais detalhtes do modelo de Árvore de Decisão na próxima trilha. Mas ele é um classificador, assim como o modelo de K-Vizinhos mais Próximos e a Regressão Logística, e o uso desse estimador segue os mesmos moldes desses estimadores. Você pode assim aplicá-lo do mesmo modo. 

In [26]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [27]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

base_estimator = DecisionTreeClassifier()

param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [ None, 3 ]}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()

DecisionTreeClassifier(criterion='entropy')

Detailed classification report:

              precision    recall  f1-score   support

      Accept       1.00      1.00      1.00        47
   Refurbish       1.00      1.00      1.00        55
      Reject       1.00      1.00      1.00        48

    accuracy                           1.00       150
   macro avg       1.00      1.00      1.00       150
weighted avg       1.00      1.00      1.00       150


